In [56]:
%matplotlib widget
import ipympl
import numpy as np
import matplotlib.pyplot as plt
import PyHippocampus as pyh
import DataProcessingTools as DPT
import os
import ipywidgets as widgets

os.chdir("/Users/chris/Downloads/picasso-misc/20181105/session01")
a = DPT.objects.processDirs(None, pyh.Unity)
os.chdir("/Users/chris/Downloads/picasso-misc/20181105/session02")
b = DPT.objects.processDirs(None, pyh.Unity)
os.chdir("/Users/chris/Downloads/picasso-misc/20181105/session3")
c = DPT.objects.processDirs(None, pyh.Unity)
plotobjs = [a,b,c]

class Main():
    def __init__(self, plotobjs, rows=None, cols=None, indexer=None):
        fig= plt.figure()
        self.fig = fig
        
        if hasattr(plotobjs, '__getitem__'):
            self.plotobjs = plotobjs
        else:
            self.plotobjs = [plotobjs]

        self.pre_button = widgets.Button(description='Pre')
        self.next_button = widgets.Button(description='Next')
        self.index_show = widgets.Text(value = currentIndex,
                         placeholder='Index',
                         description= "",
                         disabled=False)
        
        self.index_show.on_submit(self.updateIndex)
        self.pre_button.on_click(self.goprev)
        self.next_button.on_click(self.gonext)
        display(widgets.HBox([self.pre_button,self.index_show,self.next_button]))
        
        self.plotopts = [plotobj.plot(getPlotOpts=True) for plotobj in self.plotobjs]
        self.index = 0
        if cols is None:
            cols = 1
        if rows is None:
            rows = np.ceil(len(self.plotobjs)/cols)
        self.numEvents = 100_000
        
        for (i, plotobj) in enumerate(self.plotobjs):
            ax = fig.add_subplot(rows, cols, i+1)
            nn, newIdx = plotobj.plot(self.index, getNumEvents=True)
            self.numEvents = min(self.numEvents, nn)
            plotobj.plot(self.index, ax=ax, **self.plotopts[i])
            if newIdx is not None and newIdx != self.index:
                self.index_show.value = str(self.newIdx)
                self.updateIndex()
    
    def update_index(self, new_index):
        index = self.index
        if 0 <= new_index < self.numEvents:
            index = new_index
        else:
            index = self.index

        self.index = index
    
    def gonext(self,_):
        self.update_index(self.index+1)
        self.index_show.value = str(self.index)
        self.plot()
    
    def goprev(self,_):
        self.update_index(self.index-1)
        self.index_show.value = str(self.index)
        self.plot()
    
    def plot(self):
        for _ax in self.fig.axes:
            _ax.clear()
        for (i, plotobj) in enumerate(self.plotobjs):
            plotobj.plot(self.index, ax=self.fig.axes[i], **self.plotopts[i])
    
    def updateIndex(self,wdgt):
        self.update_index(int(wdgt.value))
        wdgt.value = str(self.index)
        for _ax in self.fig.axes:
            _ax.clear()
        for (i, plotobj) in enumerate(self.plotobjs):
            plotobj.plot(self.index, ax=self.fig.axes[i], **self.plotopts[i])

a = Main(plotobjs=plotobjs)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …